In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler drivertorch:  2.3 ; cuda:  cu118
detectron2: 0.6

Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from scipy.spatial import distance
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
import torch

In [3]:
from detectron2.utils.visualizer import ColorMode
from detectron2.data.datasets import register_coco_instances
import matplotlib.pyplot as plt
from detectron2.data import MetadataCatalog, DatasetCatalog

test_json = "./instance_coco_annotations_test.json" 
test_folder = "./combine_dataset/final_dataset/test/images"

register_coco_instances("terrain_test", {},test_json, test_folder)

In [5]:
output_dir  = "./output2/mask_rcnn_R_50_FPN_3x/model7"
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = output_dir
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9
cfg.INPUT.MIN_SIZE_TEST = 200
cfg.INPUT.MAX_SIZE_TEST = 1500

predictor = DefaultPredictor(cfg)

[08/12 21:51:16 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output2/mask_rcnn_R_50_FPN_3x/model7\model_final.pth ...


In [7]:
from detectron2.utils.visualizer import ColorMode
import random
import cv2

s5_metadata = MetadataCatalog.get('terrain_test')

dataset_dicts = DatasetCatalog.get('terrain_test')

dir = output_dir + "/prediction_output"

os.makedirs(dir, exist_ok=True)

counter = 1901
for d in  random.sample(dataset_dicts,counter):
    instance_json = []
    im = cv2.imread(d["file_name"])
    if im is None:
        continue
    counter = counter - 1
    # if counter == 0:
        
    #     break
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=s5_metadata, 
                   scale=1.0, 
                   instance_mode=ColorMode.IMAGE_BW )
    
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # plt.imshow(out.get_image()[:, :, ::-1]) 
    # plt.show()
    pred_dict = []
    for i in range(len(outputs['instances'])):
        instance = outputs['instances'][i]

        instance_dict = {
            "class": int(instance.pred_classes),
            "score": float(instance.scores),
            "bbox": instance.pred_boxes.tensor.to("cpu").numpy().tolist(),
            # "mask": instance.pred_masks.cpu().numpy().astype(np.uint8).tolist()
        }
        pred_dict.append(instance_dict)

    pred_masks = outputs["instances"].pred_masks.cpu().numpy()
    blank_mask = np.ones_like(im)
    for mask in pred_masks:
        blank_mask[mask] = 0  
   
    blank_mask_list = blank_mask.tolist()
    filename = d["file_name"].split("\\")[-1].replace(".jpg","")
    saved_image_file =  os.path.join(dir,filename+".png")
    print(saved_image_file)
    cv2.imwrite(saved_image_file, out.get_image()[:, :, ::-1]) 
    instance_json.append({
        "image_fname" : filename,
        "instances" : pred_dict,
        "unpredicted_coords":blank_mask_list
    })
    saved_json_file = os.path.join(dir,filename+".json")
    with open(saved_json_file,"w") as json_file:
        json.dump(instance_json,json_file,indent = 4)

[08/12 22:02:15 d2.data.datasets.coco]: Loaded 1901 images in COCO format from ./instance_coco_annotations_test.json
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\NLB_454044817EDR_F0330000NCAM00353M1.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\0050MR0002290270102792E01_DXXX.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\NLB_440995743EDR_F0240408NCAM00250M1.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\0314MR0012870750300260E01_DXXX.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\NLB_454934035EDR_F0340000NCAM00295M1.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\NLB_446597184EDR_F0280264NCAM00281M1.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\NLB_444283070EDR_F0260000NCAM00251M1.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\NLB_454666020EDR_F0331036NCAM00354M1.png
./output2/mask_rcnn_R_50_FPN_3x/model7/prediction_output\NLB_453868437EDR_F0320766NCAM00379M1.png
./output2/mas

OSError: [Errno 28] No space left on device

In [8]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.modeling import build_model
import matplotlib.pyplot as plt 

custom_val_metadata = MetadataCatalog.get("terrain_test").set(thing_classes=["sky","ridge","soil","sand","bedrock","rock","rover","trace","hole"])

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = output_dir
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75  # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9
cfg.INPUT.MIN_SIZE_TEST = 200
cfg.INPUT.MAX_SIZE_TEST = 1500
cfg.DATASETS.TEST = ("terrain_test",)
predictor = DefaultPredictor(cfg)
# Evaluation on validation dataset
evaluator = COCOEvaluator("terrain_test",cfg,False, output_dir=output_dir + "/eval/")
val_loader = build_detection_test_loader(cfg, "terrain_test")
inference_results = inference_on_dataset(predictor.model, val_loader, evaluator)


[08/13 16:26:45 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output2/mask_rcnn_R_50_FPN_3x/model7\model_final.pth ...
WARNING [08/13 16:26:46 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[08/13 16:26:49 d2.data.datasets.coco]: Loaded 1901 images in COCO format from ./instance_coco_annotations_test.json
[08/13 16:26:49 d2.data.build]: Distribution of instances among all 9 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    sky     | 155          |   ridge    | 224          |    soil    | 3712         |
|    sand    | 460          |  bedrock   | 10318        |    rock    | 4731         |
|   rover    | 30           |   trace    | 31           |    hole    | 12           |
|            |              |            |          

In [9]:
def calculate_iou(box1, box2):
    """
    Calculate IoU (Intersection over Union) between two bounding boxes.

    Parameters:
        box1 (list or numpy array): Coordinates of the first bounding box in [x1, y1, x2, y2] format.
        box2 (list or numpy array): Coordinates of the second bounding box in [x1, y1, x2, y2] format.

    Returns:
        float: Intersection over Union (IoU) value.
    """
    # Calculate coordinates of intersection area
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Calculate area of intersection
    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # Calculate area of both bounding boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # Calculate IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou


In [11]:
# Initialize counters for class-wise accuracy
iou_threshold = 0.8
metadata=  MetadataCatalog.get("terrain_test").set(thing_classes=["sky","ridge","soil","sand","bedrock","rock","rover","trace","hole"])
class_counts = {k: 0 for k in metadata.thing_classes}
class_correct = {k: 0 for k in metadata.thing_classes}

# Initialize counters for instance-wise accuracy
total_instances = 0
correct_instances = 0

# Iterate over dataset and compute accuracy
for d in dataset_dicts:
    im = cv2.imread(d["file_name"])  # Load image
    outputs = predictor(im)  # Perform inference
    instances = outputs["instances"]
    pred_classes = instances.pred_classes.cpu().numpy()
    pred_boxes = instances.pred_boxes.tensor.cpu().numpy()
    
    gt_classes = np.array([ann["category_id"] for ann in d["annotations"]])
    gt_boxes = np.array([ann["bbox"] for ann in d["annotations"]])
    
    # Compute class-wise accuracy
    for pred_class, gt_class in zip(pred_classes, gt_classes):
        if pred_class == gt_class:
            class_correct[metadata.thing_classes[pred_class]] += 1
        class_counts[metadata.thing_classes[gt_class]] += 1
    
    # Compute instance-wise accuracy
    for pred_box in pred_boxes:
        total_instances += 1
        ious = [calculate_iou(pred_box, gt_box) for gt_box in gt_boxes]
        if max(ious) >= iou_threshold:
            correct_instances += 1

# Compute and print class-wise accuracy
print("Class-wise accuracy:")
for class_name in metadata.thing_classes:
    if class_counts[class_name] == 0:
        continue
    accuracy = class_correct[class_name] / class_counts[class_name]
    print(f"{class_name}: {accuracy}")

# Compute and print instance-wise accuracy
instance_accuracy = correct_instances / total_instances
print(f"Instance-wise accuracy: {instance_accuracy}")

Class-wise accuracy:
sky: 0.4025974025974026
ridge: 0.3302752293577982
soil: 0.5893739703459637
sand: 0.4207650273224044
bedrock: 0.7556053811659192
rock: 0.7430167597765364
hole: 0.0
Instance-wise accuracy: 0.2954841511072514


In [12]:
# Function to calculate Mean Pixel Accuracy
def calculate_mean_pixel_accuracy(predictions, ground_truths):
    # Convert predictions and ground truths to numpy arrays
    predictions = np.array(predictions)
    ground_truths = np.array(ground_truths)

    # Calculate the total number of pixels
    total_pixels = np.sum(ground_truths >= 0)  # Exclude ignore label (-1)

    # Calculate the number of correct pixels
    correct_pixels = np.sum(predictions == ground_truths)

    # Calculate Mean Pixel Accuracy
    mean_pixel_accuracy = correct_pixels / total_pixels

    return mean_pixel_accuracy


In [14]:
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog, build_detection_test_loader
import numpy as np


# Lists to store predictions and ground truths
predictions = []
ground_truths = []

# Iterate over the dataset and make predictions
for d in dataset_dicts:
    # Perform inference
    with torch.no_grad():
        outputs = predictor(batch)

    # Get predicted segmentation masks and ground truth masks
    pred_masks = outputs["instances"].pred_masks.cpu().numpy()
    gt_masks = batch["instances"].gt_masks.cpu().numpy()

    # Append to the lists
    predictions.extend(pred_masks)
    ground_truths.extend(gt_masks)

# Calculate Mean Pixel Accuracy
mean_pixel_accuracy = calculate_mean_pixel_accuracy(predictions, ground_truths)

# Print the result
print("Mean Pixel Accuracy (mAcc):", mean_pixel_accuracy)


AttributeError: 'dict' object has no attribute 'shape'